# Homework on MLE


#####   Estimate a Normal-Half Normal Model.

Consider the Model A below:

\begin{aligned}
 y_i & = \alpha + x_i' \beta + v_i - u_i,\\
 v_i & \sim N(0, \sigma_v^2), \\
 u_i & \sim N^+(0, \sigma_u^2).
\end{aligned}
 
 
The following is Model A's log-likelihood function of the $i$th observation. 

\begin{align}
L_i = - \ln \left(\frac{1}{2}\right) -\frac{1}{2}\ln (\sigma_v^2 + \sigma_u^2) + \ln
\phi\left(\frac{\epsilon_i}{\sqrt{\sigma_v^2 + \sigma_u^2}} \right) +
\ln \Phi\left(\frac{\mu_{*i}}{\sigma_*} \right),
\end{align}

where $\phi(z)$ and $\Phi(z)$ are the PDF and CDF of a standard normal distribution. Also,

\begin{aligned}
 \mu_{*i}  = \frac{-\sigma_u^2 \epsilon_i}{\sigma_v^2 + \sigma_u^2},\qquad
 \sigma_*^2  = \frac{\sigma_v^2  \sigma_u^2}{\sigma_v^2 + \sigma_u^2}. 
\end{aligned}


The attached dataset, `sampledata.csv`, contains data of agricultural production from India. The variables are the follows. They have been converted to appropriate units (taking log, etc.) so no further data processing is necessary.


|          |  |        |
|-------------------------------------|---|---------------------|
| yvar: rice output                   |   | Lland: land         |
| Plland: irrigated land              |   | Llabor: labor       |
| Lbull: bull cost                    |   | Lcost: other costs  |
| yr: production year                 |   | age: age of farmers |
| school: farmers' years of schooling |   | yr_1: same as year  |


The Indian farming model is the follows.

$yvar_i = \alpha + \beta_1 Lland_i + \beta_2 Plland_i + \beta_3 Llabor_i + \beta_4 Lbull_i + \beta_5 Lcost_i + \beta_6 yr_i + v_i - u_i$.
  
You may use the following code to read in the data.

```julia
####################
using DataFrames, CSV
df = DataFrame(CSV.File("sampledata.csv"))
y = df[:, "yvar"]       # the dep var
x = Matrix(df[:, 2:7])  # the indep vars, not including a constant
####################
```

###### Your work: 

- Based on (1), write a program of Model A's log-likelihood function and call it `NHN_mle`. You may use `loglike5` in the lecture note as the reference. 


- Use `NHN_mle` to estimate the Indian farming model. You may conduct the estimation using `Optim` or your own programs. The required result is a table with three columns: the 1st column is the coefficients, the 2nd column is the standard errors, and the 3rd column is the $t$ statistics. They do not have to be in DataFrames.


- The Estimation Guidelines:
  
  - You may use $-0.1$ as the initial values for all parameters. Or you may use the OLS result as the initial values for the $\alpha$ and $\beta$ coefficients (`x2=hcat(ones(size(y,1), 1), x); ols=inv(x2'x2)*(x2'y)`). Or, you may choose any initial values that seem to be reasonable choices. However, you _**cannot**_ use the true answer as the initial values.

  - I strongly suggest that your program uses the `autodiff = :forward` option (which uses automatic differentiation) in the estimation.
  
    - The `autodiff = :forward` option puts stringent requirements on data types which may not easy to work out. I suggest that you start with your program without the option (which would then default to numerical finite differences that is easier on the data). After you have a working program, you may add the option back and see if it works. Most likely you'll have error messages and you have to work out the issues. If you can't make it work, that's fine. Try your best.  
  
- Hint: Look at $|g(x)|$ to judge whether it is converged. It should be smaller than the convergence criterion. 

In [ ]:
#import Pkg; Pkg.add("CSV")

In [1]:
using Distributions, DataFrames, CSV, LinearAlgebra, ForwardDiff, Optim

In [2]:
df = DataFrame(CSV.File("HWK13sampledata.csv"))
y = df[:, "yvar"]       # the dep var
x = Matrix(df[:, 2:7])

271×6 Matrix{Float64}:
 -0.210721  0.0       5.26269  4.41884  0.0       6.0
 -0.210721  0.0       4.84419  4.20469  0.0       7.0
 -0.210721  0.0       4.39445  3.78419  0.0       8.0
 -0.210721  0.0       4.79579  4.23411  4.51501   9.0
 -0.211476  0.0       5.54908  4.95583  4.31431  10.0
  0.482426  0.0       5.53733  5.06259  0.0       6.0
  0.19062   0.0       5.17048  4.58497  0.0       7.0
  0.482426  0.0       5.57595  4.86754  0.0       9.0
  0.481671  0.0       5.38907  4.96981  0.0      10.0
 -0.210721  0.0       4.31749  3.68888  0.0       7.0
 -0.916291  0.0       4.07754  3.43399  0.0       8.0
 -0.210721  0.0       5.96101  4.02535  3.76898   6.0
 -0.210721  0.0       5.47646  4.64439  3.8907    7.0
  ⋮                                               ⋮
  1.58858   0.669422  8.49208  6.82546  7.15436  10.0
  2.06051   0.168153  8.52258  7.06133  7.66435   1.0
  2.09924   0.11152   8.70814  7.32251  7.11658   2.0
  1.63705   0.315175  8.16877  6.63857  6.88085   3.0
  1.699

In [83]:
function NHN_mle(y, x, α, β, log_σv², log_σu²)  
    
    σv² = exp(log_σv²)
    σu² = exp(log_σu²)
    PDF(ϵ) = pdf(Normal(0,1),ϵ)
    CDF(ϵ) = cdf(Normal(0,1),ϵ)
    sigma = (σv²*σu²)/(σv²+σu²)
    sigma = sqrt(sigma)
    mu(ϵ) = ((-σu²*ϵ)/(σv²+σu²))
    ϵ  = y .- α .- x*β
    
    llike = 0.0          
    for i in eachindex(ϵ)
        #llike += -log(1/2)-(log(σu²+σv²))/2 + log(PDF(ϵ[i]/(sqrt(σv²+σu²)))) + log(CDF(mu(ϵ[i])/sigma))
        llike += -log(1/2)-(log(σu²+σv²))/2 + log(pdf(Normal(0,1), ϵ[i]/(sqrt(σv²+σu²)))) + log(cdf(Normal(0,1), mu(ϵ[i])/sigma))
    end
    
    return llike = -llike
end

NHN_mle (generic function with 1 method)

In [111]:
nofxvar = 6
x2 = hcat(ones(size(y,1), 1), x)
ols=inv(x2'x2)*(x2'y)
ols=push!(ols,1,1)
func = TwiceDifferentiable(vars -> NHN_mle(y, x, vars[1], vars[2:nofxvar+1], vars[end-1], vars[end]), ones(nofxvar+3))
res = optimize(func, ols , Newton())
print(res,"\n")
coevec = Optim.minimizer(res)
rescoef = deepcopy(coevec)                
rescoef[end] = exp(coevec[end])
rescoef[end - 1] = exp(_coevec[end - 1]) 

hess  = Optim.hessian!(func, coevec)  

varcov_matrix = inv(hess)
stderror = []
stderror = sqrt.(diag(varcov_matrix))

stderror[end] = rescoef[end] * stderror[end]
stderror[end-1] = rescoef[end-1] * stderror[end-1]
tstats = rescoef ./ stderror

table = hcat(rescoef, stderror, tstats)
println("The estimation table is")
table |> display

 * Status: success (objective increased between iterations)

 * Candidate solution
    Final objective value:     9.336604e+01

 * Found with
    Algorithm:     Newton's Method

 * Convergence measures
    |x - x'|               = 9.90e-11 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.89e-11 ≰ 0.0e+00
    |f(x) - f(x')|         = 5.68e-14 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 6.09e-16 ≰ 0.0e+00
    |g(x)|                 = 7.04e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    14
    f(x) calls:    40
    ∇f(x) calls:   40
    ∇²f(x) calls:  14

The estimation table is


9×3 Matrix{Float64}:
  1.59012     0.349984     4.54342
  0.286194    0.0698685    4.09618
  0.234652    0.176519     1.32933
  1.15546     0.0812037   14.2292
 -0.421433    0.0594057   -7.09415
  0.00661819  0.0130431    0.507411
  0.0340904   0.00803052   4.2451
  1.25733     0.353605     3.55574
  0.25651     0.0421415    6.08686